# Лабораторная работа №7. Рекуррентные нейронные сети для анализа текста

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Задание 1.
Загрузите данные. Преобразуйте текстовые файлы во внутренние структуры данных, которые используют индексы вместо слов.

После выполнения скрипта в директории movie_data будет 4 файла:
full_test_neg.txt,
full_test_pos.txt,
full_train_neg.txt,
full_train_pos.txt

In [49]:
import os

dataset_path = os.path.join('data', 'aclImdb', 'movie_data')
def read_reviews(filename):
    reviews = []
    with open(os.path.join(dataset_path, filename)) as f:
        for review in f.readlines():
            reviews.append(review.strip())
    
    return reviews


pos_train_reviews = read_reviews('full_train_pos.txt')
neg_train_reviews = read_reviews('full_train_neg.txt')
pos_test_reviews = read_reviews('full_test_pos.txt')
neg_test_reviews = read_reviews('full_test_neg.txt')

In [50]:
len(pos_test_reviews), len(neg_test_reviews), len(pos_test_reviews), len(neg_test_reviews)

(12500, 12500, 12500, 12500)

In [51]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def clean_up(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews


pos_train_reviews = clean_up(pos_train_reviews)
neg_train_reviews = clean_up(neg_train_reviews)
pos_test_reviews = clean_up(pos_test_reviews)
neg_test_reviews = clean_up(neg_test_reviews)

In [80]:
y_train = np.array(len(pos_train_reviews) * [1] + len(neg_train_reviews) * [0])
y_test = np.array(len(pos_test_reviews) * [1] + len(neg_test_reviews) * [0])
y_train.shape, y_test.shape

((25000,), (25000,))

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, split=' ')
tokenizer.fit_on_texts(pos_train_reviews + neg_train_reviews)

In [74]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(pos_train_reviews + neg_train_reviews + pos_test_reviews + neg_test_reviews)
X = pad_sequences(X)

In [76]:
X.shape

(50000, 2176)

In [77]:
X_train = X[:25000]
X_test = X[25000:]

In [81]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25000, 2176), (25000, 2176), (25000,), (25000,))

In [82]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

## Задание 2.
Реализуйте и обучите двунаправленную рекуррентную сеть (LSTM или GRU). Какого качества классификации удалось достичь?

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

network = Sequential([
    Embedding(5000, 128, input_length = X_train.shape[1]),
    Bidirectional(LSTM(196)),
    Dense(2, activation = 'softmax')
])

network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2176, 128)         640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 392)               509600    
_________________________________________________________________
dense (Dense)                (None, 2)                 786       
Total params: 1,150,386
Trainable params: 1,150,386
Non-trainable params: 0
_________________________________________________________________


In [93]:
network.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
network.fit(X_train, y_train, epochs = 7, batch_size = 128, verbose = 2)

Train on 20000 samples
Epoch 1/7


InvalidArgumentError:  indices[56,2133] = 5793 is not in [0, 5000)
	 [[node sequential/embedding_3/embedding_lookup (defined at <ipython-input-93-a014f4ba37c7>:2) ]] [Op:__inference_distributed_function_9849]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding_3/embedding_lookup:
 sequential/embedding_3/embedding_lookup/7830 (defined at /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/contextlib.py:112)

Function call stack:
distributed_function


## Задание 3.
Используйте индексы слов и их различное внутреннее представление (word2vec, glove). Как влияет данное преобразование на качество классификации?

## Задание 4.
Поэкспериментируйте со структурой сети (добавьте больше рекуррентных, полносвязных или сверточных слоев). Как это повлияло на качество классификации?


## Задание 5.
Используйте предобученную рекуррентную нейронную сеть (например, DeepMoji или что-то подобное).